### modeling the hdv dataset ###

In [9]:
!bq --location=US mk --dataset hdv_modeled

BigQuery error in mk operation: Dataset 'alert-result-266803:hdv_modeled'
already exists.


First, we create a Jurisdiction table composed of states, county names, and FIPS codes (numerical equivalent to county).

In [16]:
%%bigquery
create table hdv_modeled.Jurisdiction as 
select distinct state, jurisdiction as jname, fipscode
from hdv_staging.House2016
union distinct
select distinct state, jurisdiction as jname, fipscode
from hdv_staging.House2018
union distinct
select distinct state, jurisdiction as jname, fipscode
from hdv_staging.Senate2016
union distinct
select distinct state, jurisdiction as jname, fipscode
from hdv_staging.Senate2018
union distinct
select distinct state, jurisdiction as jname, fipscode
from hdv_staging.President2016

""


Now, we need to check for the primary key. From Milestone 3, we noticed that some jurisdictions are represented with the same fipscode. This is an issue we will rectify when cleaning the data. For now, we propose that state, jname, and fipscode are the primary keys.

In [25]:
%%bigquery
select count(*)
from hdv_modeled.Jurisdiction;

,f0_
0,6601


In [27]:
%%bigquery
select count(distinct concat(state,jname,fipscode))
from hdv_modeled.Jurisdiction

,f0_
0,6601


Next, we create an Election table featuring office and year.

In [17]:
%%bigquery
create table hdv_modeled.Election as 
select distinct office, year 
from hdv_staging.House2016
union distinct
select distinct office, year
from hdv_staging.House2018
union distinct
select distinct office, year
from hdv_staging.Senate2016
union distinct
select distinct office, year
from hdv_staging.Senate2018
union distinct
select distinct office, year
from hdv_staging.President2016

""


Now, we need to establish a primary key. We propose the combination of office and year as the primary key.

In [29]:
%%bigquery
select count(*)
from hdv_modeled.Election

,f0_
0,6


In [28]:
%%bigquery
select count(distinct concat(office,year))
from hdv_modeled.Election

,f0_
0,6


Our last entity table is Candidate. This includes names and party affilitations.

In [18]:
%%bigquery
create table hdv_modeled.Candidate as 
select distinct candidate as cname, party 
from hdv_staging.House2016
union distinct
select distinct candidate as cname, party
from hdv_staging.House2018
union distinct
select distinct candidate as cname, party
from hdv_staging.Senate2016
union distinct
select distinct candidate as cname, party
from hdv_staging.Senate2018
union distinct
select distinct candidate as cname, party
from hdv_staging.President2016

""


Now, we establish the primary key: the combination of name and party.

In [31]:
%%bigquery
select count(*) from hdv_modeled.Candidate

,f0_
0,2773


In [36]:
%%bigquery
select count(distinct concat(party, cname))
from hdv_modeled.Candidate

,f0_
0,2439


This indicates that there may be duplicates in our candidates. This is an issue we may have to address in our data cleaning, if present:

In [33]:
%%bigquery
select c.cname, c.party, count(*)
from hdv_modeled.Candidate c
GROUP BY c.cname, c.party
HAVING count(*) > 1

,cname,party,f0_


There does not seem to be any duplicates after all! The real error is that our previous query does not detect null fields for candidate. We claim that the discrepancy between 2773 and 2439 (2773-2439 = 334) is in null fields for candidates:

In [58]:
%%bigquery
select count(*)
from hdv_modeled.Candidate
where party is null

,f0_
0,334


Lastly, we create a junction table Results between Candidate, Election, and Jurisdiction. All 3 tables have m:n relationships with one another.

In [24]:
%%bigquery
create table hdv_modeled.Results as 
select distinct state, jurisdiction as jname, fipscode, candidate as cname, party, office, year, votes, total_votes
from hdv_staging.House2016
union distinct
select distinct state, jurisdiction as jname, fipscode, candidate as cname, party, office, year, votes, total_votes
from hdv_staging.House2018
union distinct
select distinct state, jurisdiction as jname, fipscode, candidate as cname, party, office, year, votes, total_votes
from hdv_staging.Senate2016
union distinct
select distinct state, jurisdiction as jname, fipscode, candidate as cname, party, office, year, votes, total_votes
from hdv_staging.Senate2018
union distinct
select distinct state, jurisdiction as jname, fipscode, candidate as cname, party, office, year, votes, total_votes
from hdv_staging.President2016
order by year, office, state, jname, votes desc, cname

""


Having created the junction table, we attempt to define a primary key among the many foreign keys present.

In [41]:
%%bigquery
select count(*)
from hdv_modeled.Results

,f0_
0,121227


In [56]:
%%bigquery
select count(distinct concat(cname, state, jname, office, year, votes, total_votes))
from hdv_modeled.Results

,f0_
0,121227


The Results table dictates a primary key consisting of cname, state, jname, office, year, votes, total_votes. Some candidates do not have a party affiliation (i.e. null), which necessitates the inclusion of votes and total_votes in primary key.

### Beam Transforms ###

In [1]:
%run Jurisdiction_beam.py

/home/jupyter/.local/lib/python3.5/site-packages/apache_beam/io/gcp/bigquery.py:1421: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
INFO:apache_beam.runners.direct.direct_runner:Running pipeline with DirectRunner.
INFO:apache_beam.internal.gcp.auth:Setting socket default timeout to 60 seconds.
INFO:apache_beam.internal.gcp.auth:socket default timeout is 60.0 seconds.
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:apache_beam.io.gcp.bigquery_tools:Using location 'US' from table <TableReference
 datasetId: 'hdv_modeled'
 projectId: 'alert-result-266803'
 tableId: 'Jurisdiction'> referenced by query SELECT state, jname, fipscode FROM hdv_modeled.Jurisdiction where state = "NH" order by state, jname
INFO:apache_beam.io.filebasedsink:Starting finalize_write threads with num_shards: 1 (skipped: 0), batches: 1,

Now, we establish the primary key for the Jurisdiction_Beam table: the composite of state and jname

In [2]:
%%bigquery
select count(*)
from hdv_modeled.Jurisdiction_Beam

,f0_
0,315


In [3]:
%%bigquery
select count(distinct concat(state,jname))
from hdv_modeled.Jurisdiction_Beam

,f0_
0,315


Now, we check whether this table has a FK relationship with the table from our potential secondary dataset: education. The suspected foreign key is fipscode, which is the numerical equivalent of each jurisdiction.

In [4]:
%%bigquery
select count(*) from hdv_modeled.Jurisdiction_Beam jb left join education.education e
on jb.fipscode = e.FIPS_Code where e.FIPS_Code is null

,f0_
0,0


At least for the subset of data that is transformed, fipscode is indeed a foreign key.

Now, we turn our attention to the full Jurisdiction table.

In [17]:
%run Jurisdiction_beam_dataflow.py

/home/jupyter/.local/lib/python3.5/site-packages/apache_beam/runners/dataflow/dataflow_runner.py:740: BeamDeprecationWarning: BigQuerySink is deprecated since 2.11.0. Use WriteToBigQuery instead.
  kms_key=transform.kms_key))


Now, we check the primary key, as before.

In [2]:
%%bigquery
select count(*)
from hdv_modeled.Jurisdiction_Beam_DF

,f0_
0,4713


In [3]:
%%bigquery
select count(distinct concat(state,jname))
from hdv_modeled.Jurisdiction_Beam_DF

,f0_
0,4713


As expected, state and jname are appropriate primary keys. Now, we see if this full table has fipscode as a foreign key.

In [4]:
%%bigquery
select count(*) from hdv_modeled.Jurisdiction_Beam_DF jb left join education.education e
on jb.fipscode = e.FIPS_Code where e.FIPS_Code is null

,f0_
0,40


In [6]:
%%bigquery
select jb.state, jb.jname, jb.fipscode from hdv_modeled.Jurisdiction_Beam_DF jb left join education.education e
on jb.fipscode = e.FIPS_Code where e.FIPS_Code is null order by jb.state, jb.jname

,state,jname,fipscode
0,AK,District 1,2901
1,AK,District 10,2910
2,AK,District 11,2911
3,AK,District 12,2912
4,AK,District 13,2913
5,AK,District 14,2914
6,AK,District 15,2915
7,AK,District 16,2916
8,AK,District 17,2917
9,AK,District 18,2918


All of the foreign key violations occur in Alaska; the electoral districts reported in this data do not correspond with the traditional FIPS codes for Alaska. We may have to disregard Alaska in our analysis or convert these FIPS code values to their conventional forms -- although there may not be a straightforward way of doing so, especially if an electoral district overlaps multiple counties. Considering the tables' current form, fipscode is not a legitimate foreign key.

Now, we transform the Candidate table.

In [1]:
%run Candidate_beam.py

/home/jupyter/.local/lib/python3.5/site-packages/apache_beam/io/gcp/bigquery.py:1421: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
INFO:apache_beam.runners.direct.direct_runner:Running pipeline with DirectRunner.
INFO:apache_beam.internal.gcp.auth:Setting socket default timeout to 60 seconds.
INFO:apache_beam.internal.gcp.auth:socket default timeout is 60.0 seconds.
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:apache_beam.io.gcp.bigquery_tools:Using location 'US' from table <TableReference
 datasetId: 'hdv_modeled'
 projectId: 'alert-result-266803'
 tableId: 'Candidate'> referenced by query SELECT cname, party FROM hdv_modeled.Candidate limit 50
INFO:apache_beam.io.gcp.bigquery_tools:Created table alert-result-266803.hdv_modeled.Candidate_Beam with schema <TableSchema
 fields: [<TableFieldSchema
 fiel

We claim that the combination of cname and party forms the primary key for this table.

In [2]:
%%bigquery
select count(*)
from hdv_modeled.Candidate_Beam

,f0_
0,49


In [3]:
%%bigquery
select count(distinct concat(cname,party))
from hdv_modeled.Candidate_Beam

,f0_
0,49


Since this preliminary result is acceptable, we move to the full Candidate table.

In [5]:
%run Candidate_beam_dataflow.py

/home/jupyter/.local/lib/python3.5/site-packages/apache_beam/runners/dataflow/dataflow_runner.py:740: BeamDeprecationWarning: BigQuerySink is deprecated since 2.11.0. Use WriteToBigQuery instead.
  kms_key=transform.kms_key))
INFO:apache_beam.runners.dataflow.internal.apiclient:Starting GCS upload to gs://poutine_bucket/staging/student-df3.1583644005.658830/pipeline.pb...
INFO:apache_beam.runners.dataflow.internal.apiclient:Completed GCS upload to gs://poutine_bucket/staging/student-df3.1583644005.658830/pipeline.pb in 0 seconds.
INFO:apache_beam.runners.portability.stager:Downloading source distribution of the SDK from PyPi
INFO:apache_beam.runners.portability.stager:Executing command: ['/usr/bin/python3', '-m', 'pip', 'download', '--dest', '/tmp/tmp877ntl53', 'apache-beam==2.19.0', '--no-deps', '--no-binary', ':all:']
INFO:apache_beam.runners.portability.stager:Staging SDK sources from PyPI to gs://poutine_bucket/staging/student-df3.1583644005.658830/dataflow_python_sdk.tar
INFO:apac

We select the combination of cname and party to be the primary key.

In [6]:
%%bigquery
select count(*)
from hdv_modeled.Candidate_Beam_DF

,f0_
0,2404


In [7]:
%%bigquery
select count(distinct concat(cname,party))
from hdv_modeled.Candidate_Beam_DF

,f0_
0,2404


The results indicate that this is indeed the primary key.

Finally, we transform the Results table. This is a child of all 3 other tables (Candidate, Election, Jurisdiction), so we will also test for foreign key violations.

In [6]:
%run Results_beam.py

/home/jupyter/.local/lib/python3.5/site-packages/apache_beam/io/gcp/bigquery.py:1421: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
INFO:apache_beam.runners.direct.direct_runner:Running pipeline with DirectRunner.
INFO:apache_beam.io.gcp.bigquery_tools:Using location 'US' from table <TableReference
 datasetId: 'hdv_modeled'
 projectId: 'alert-result-266803'
 tableId: 'Results'> referenced by query SELECT state, jname, fipscode, cname, party, office, year, votes, total_votes FROM hdv_modeled.Results WHERE state = "DE" 
INFO:apache_beam.io.gcp.bigquery_tools:Created table alert-result-266803.hdv_modeled.Results_Beam with schema <TableSchema
 fields: [<TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'state'
 type: 'STRING'>, <TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'jname'
 type: 'STRING'>, <TableFieldSchema
 fields: []
 m

Now we check that the primary key (state,jname,cname,party,office,year) is valid

In [7]:
%%bigquery
select count(*)
from hdv_modeled.Results_Beam

,f0_
0,60


In [8]:
%%bigquery
select count(distinct concat(state,jname,cname,party,office,year))
from hdv_modeled.Results_Beam

,f0_
0,60


Now, we need to check that there are no foreign key violations:

In [9]:
%%bigquery
select count(*) from hdv_modeled.Results_Beam rb left join hdv_modeled.Jurisdiction_Beam_DF jb
on rb.state = jb.state where jb.state is null

,f0_
0,0


In [10]:
%%bigquery
select count(*) from hdv_modeled.Results_Beam rb left join hdv_modeled.Jurisdiction_Beam_DF jb
on rb.jname = jb.jname where jb.jname is null

,f0_
0,0


In [11]:
%%bigquery
select count(*) from hdv_modeled.Results_Beam rb left join hdv_modeled.Jurisdiction_Beam_DF jb
on rb.fipscode = jb.fipscode where jb.fipscode is null

,f0_
0,0


In [12]:
%%bigquery
select count(*) from hdv_modeled.Results_Beam rb left join hdv_modeled.Candidate_Beam_DF jb
on rb.cname = jb.cname where jb.cname is null

,f0_
0,0


In [13]:
%%bigquery
select count(*) from hdv_modeled.Results_Beam rb left join hdv_modeled.Candidate_Beam_DF jb
on rb.party = jb.party where jb.party is null

,f0_
0,0


In [14]:
%%bigquery
select count(*) from hdv_modeled.Results_Beam rb left join hdv_modeled.Election jb
on rb.office = jb.office where jb.office is null

,f0_
0,0


In [15]:
%%bigquery
select count(*) from hdv_modeled.Results_Beam rb left join hdv_modeled.Election jb
on rb.year = jb.year where jb.year is null

,f0_
0,0


Lastly, we extend this analysis to the full table.

In [8]:
%run Results_beam_dataflow.py

/home/jupyter/.local/lib/python3.5/site-packages/apache_beam/runners/dataflow/dataflow_runner.py:740: BeamDeprecationWarning: BigQuerySink is deprecated since 2.11.0. Use WriteToBigQuery instead.
  kms_key=transform.kms_key))
INFO:apache_beam.runners.dataflow.internal.apiclient:Starting GCS upload to gs://poutine_bucket/staging/student-df2.1583644625.752116/pipeline.pb...
INFO:apache_beam.runners.dataflow.internal.apiclient:Completed GCS upload to gs://poutine_bucket/staging/student-df2.1583644625.752116/pipeline.pb in 0 seconds.
INFO:apache_beam.runners.portability.stager:Downloading source distribution of the SDK from PyPi
INFO:apache_beam.runners.portability.stager:Executing command: ['/usr/bin/python3', '-m', 'pip', 'download', '--dest', '/tmp/tmp3i2_1nfx', 'apache-beam==2.19.0', '--no-deps', '--no-binary', ':all:']
INFO:apache_beam.runners.portability.stager:Staging SDK sources from PyPI to gs://poutine_bucket/staging/student-df2.1583644625.752116/dataflow_python_sdk.tar
INFO:apac

We check the primary key:

In [9]:
%%bigquery
select count(*)
from hdv_modeled.Results_Beam_DF

,f0_
0,109182


In [10]:
%%bigquery
select count(distinct concat(state,jname,cname,party,office,year))
from hdv_modeled.Results_Beam_DF

,f0_
0,109182


Lastly, we check the foreign key constraints:

In [11]:
%%bigquery
select count(*) from hdv_modeled.Results_Beam_DF rb left join hdv_modeled.Jurisdiction_Beam_DF jb
on rb.state = jb.state where jb.state is null

,f0_
0,0


In [12]:
%%bigquery
select count(*) from hdv_modeled.Results_Beam_DF rb left join hdv_modeled.Jurisdiction_Beam_DF jb
on rb.jname = jb.jname where jb.jname is null

,f0_
0,0


In [13]:
%%bigquery
select count(*) from hdv_modeled.Results_Beam_DF rb left join hdv_modeled.Jurisdiction_Beam_DF jb
on rb.fipscode = jb.fipscode where jb.fipscode is null

,f0_
0,0


In [14]:
%%bigquery
select count(*) from hdv_modeled.Results_Beam_DF rb left join hdv_modeled.Candidate_Beam_DF jb
on rb.cname = jb.cname where jb.cname is null

,f0_
0,0


In [15]:
%%bigquery
select count(*) from hdv_modeled.Results_Beam_DF rb left join hdv_modeled.Candidate_Beam_DF jb
on rb.party = jb.party where jb.party is null

,f0_
0,0


In [16]:
%%bigquery
select count(*) from hdv_modeled.Results_Beam_DF rb left join hdv_modeled.Election jb
on rb.office = jb.office where jb.office is null

,f0_
0,0


In [17]:
%%bigquery
select count(*) from hdv_modeled.Results_Beam_DF rb left join hdv_modeled.Election jb
on rb.year = jb.year where jb.year is null

,f0_
0,0
